# Ensemble fire probability, frequency, and size forecasts using SEAS5

Author: Jatan Buch (v1, 230830) || (v2, 240822)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import random
from numpy.lib.stride_tricks import sliding_window_view
from scipy import stats, interpolate
from scipy.optimize import minimize
from scipy.special import gamma
from sklearn.linear_model import LinearRegression
from math import factorial
import itertools
from copy import deepcopy

import netCDF4 # module that reads in .nc files (built on top of HDF5 format)
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import xarray
import rioxarray

from shapely.geometry import Point, mapping
from shapely.geometry.polygon import Polygon
from pyproj import CRS, Transformer # for transforming projected coordinates to elliptical coordinates
import cartopy.crs as ccrs # for defining and transforming coordinate systems
import cartopy.feature as cfeature # to add features to a cartopy map
import cartopy.io.shapereader as shpreader

#self-libraries
from fire_utils import *
from ml_utils import *
from ssf_utils import *

#modules for data processing and gradient boosting 
from ngboost import NGBRegressor, distns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, SplineTransformer
from sklearn.metrics import mean_squared_error, r2_score
import pickle # for saving and loading models
from pathlib import Path
import shap

#modules for neural network
import tensorflow as tf
import tensorflow_probability as tfp
tfd= tfp.distributions

from datetime import datetime, timedelta
from cftime import num2date, date2num, DatetimeGregorian
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import matplotlib.path as mpltPath
from plot_params_mpl39 import params
import matplotlib.pylab as pylab
import seaborn as sns
pylab.rcParams.update(params)
cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

%matplotlib inline
%config IPython.matplotlib.backend = 'retina'
%config InlineBackend.figure_format = 'retina'

## Generating output files

In [2]:
fire_activity_ensemble_ssf(tot_ens_mems= 51, target_yr= 2024, firemon_pred_flag= 'dynamical_forecasts', fcast_type= 'jaso', pred_var_arr= ['Tmax', 'Prec', 'VPD', 'Tmin'], \
                                                                                                        freq_id= '08_07_23', seed= 654, size_id= '08_21_23', debug= False)

  0%|          | 0/51 [00:00<?, ?it/s]

747/747 [==============================] - 0s 265us/step
Saved fire probability xarray for 2024


19/19 [==============================] - 0s 367us/step


10/10 [==============================] - 0s 422us/step


17/17 [==============================] - 0s 330us/step


...

1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 7ms/step


2/2 [==============================] - 0s 844us/step


1/1 [==============================] - 0s 7ms/step


100%|██████████| 51/51 [37:20<00:00, 43.93s/it]


## Creating ensemble baselines

In [4]:
rescale_flag= True
sys_no= None
clim_df= pd.read_hdf('../data/clim_fire_freq_12km_w2022_rescaled_data.h5')
clim_df.loc[clim_df[clim_df.fire_freq > 1].index, 'fire_freq']= np.ones(len(clim_df[clim_df.fire_freq > 1].index), dtype= np.int64)

ens_no_arr= np.arange(0, 25, 1)
target_yr_arr= np.arange(2001, 2020, 1)
pred_var_arr= ['Tmax', 'Prec', 'VPD', 'Tmin'] #'VPD', 'FFWI',
firemon_pred_flag= 'dynamical_forecasts' #'dynamical_forecasts' #'observations'
pred_drop_cols= ['SWE_mean', 'SWE_max', 'AvgSWE_3mo']

In [6]:
for target_yr in target_yr_arr:
    for ens_no in ens_no_arr:
        pred_mon_arr=  np.array([460, 461, 462, 463, 464]) - (2022 - target_yr)*12
        X_pred_ur_df, X_pred_test_df= fire_pred_df_func(clim_df, target_yr, pred_mon_arr, pred_var_arr, firemon_pred_flag, sys_no= sys_no, ens_no= ens_no, freq_flag= 'prediction') 
        fire_prob_pred_func(freq_id= '08_07_23', seed= 654, X_tot_df= X_pred_ur_df, X_test_df= X_pred_test_df, pred_mon_arr= pred_mon_arr, sav_flag= True, target_year= target_yr, \
                                                                                                                            firemon_pred_flag= 'dynamical_forecasts', ens_no= ens_no)

100%|██████████| 2/2 [01:43<00:00, 51.69s/it]


747/747 [==============================] - 0s 350us/step
Saved fire probability xarray for 2001


100%|██████████| 2/2 [01:39<00:00, 49.55s/it]


747/747 [==============================] - 0s 364us/step
Saved fire probability xarray for 2001


100%|██████████| 2/2 [01:44<00:00, 52.12s/it]


...

747/747 [==============================] - 0s 329us/step
Saved fire probability xarray for 2019


100%|██████████| 2/2 [01:37<00:00, 48.51s/it]


747/747 [==============================] - 0s 337us/step
Saved fire probability xarray for 2019


100%|██████████| 2/2 [01:38<00:00, 49.30s/it]


747/747 [==============================] - 0s 335us/step
Saved fire probability xarray for 2019
